In [1]:
#import packages
import os
import glob
from nipype.interfaces.io import DataGrabber, DataSink, SelectFiles # Data i/o
from nipype.interfaces.utility import IdentityInterface     # utility
from nipype.pipeline.engine import Node, Workflow        # pypeline engine
from nipype.interfaces.nipy.preprocess import Trim

from nipype.interfaces.fsl import SliceTimer, FSLCommand, Reorient2Std, MCFLIRT, TemporalFilter, FLIRT, maths
from nipype.interfaces.freesurfer import Resample, Binarize
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from nipype.workflows.fmri.fsl.preprocess import create_susan_smooth
#from nipype.workflows.rsfmri.fsl.resting import 

#import nipype.algorithms.modelgen as model   # model generation
#import nipype.algorithms.rapidart as ra      # artifact detection

#set output file type for FSL to NIFTI
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
studyhome = '/Volumes/iang/active/BABIES/BABIES_rest/pypelineDev'
raw_data = studyhome + '/raw'
output_dir = studyhome + '/processed/preproc'
#template_brain = '/Volumes/iang/active/BABIES/infant_templates/nihpd_05-08mo_brain/nihpd_0508_t2w_brain_2mm.nii'

subjects_list = ['035-BABIES-T1']#os.listdir(raw_data)
vols_to_trim = 4
interleave = False
TR = 2 # in seconds
slice_dir = 3 # 1=x, 2=y, 3=z
proc_cores = 4 # number of cores of processing for the workflows
resampled_voxel_size = (2,2,2)
fwhm = 4 #fwhm for smoothing with SUSAN
mask_dilation = 1

/usr/local/lib/python2.7/site-packages/nipy/labs/glm/glm.py:9: FutureWarning: Module nipy.labs.utils.routines deprecated, will be removed
  from ..utils import mahalanobis
/usr/local/lib/python2.7/site-packages/nipype/interfaces/nipy/model.py:18: FutureWarning: Module nipy.labs.glm deprecated, will be removed. Please use nipy.modalities.fmri.glm instead.
  import nipy.labs.glm.glm as GLM


In [2]:
## File handling Nodes

# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id','fwhm']),
                     name="infosource")
infosource.iterables = ('subject_id', subjects_list)


# Data grabber- select fMRI and sMRI
templates = {'struct': raw_data + '/{subject_id}/anat/skullstripped_anat.nii',
            'func': raw_data + '/{subject_id}/rest/rest_raw.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

In [8]:
## Nodes for preprocessing

# Reorient to standard space using FSL
reorientfunc = Node(Reorient2Std(), name='reorientfunc')
reorientstruct = Node(Reorient2Std(), name='reorientstruct')

# Trim first 4 volumes using nipype 
trimvols = Node(Trim(begin_index=vols_to_trim), name='trimvols')

#Slice timing correction based on interleaved acquisition using FSL
slicetime_correct = Node(SliceTimer(interleaved=interleave, slice_direction=slice_dir,
                                   time_repetition=TR),
                            name='slicetime_correct')

# Motion correction- MEL
motion_correct = Node(MCFLIRT(), name='motion_correct')

# Bias Correction- input image is 'input_image' output is 'output_image'
## need ot make additional workflow
#n4_correct = Node(N4BiasFieldCorrection(save_bias=True), 
#                  name='n4_correct')

# Reslice- using MRI_convert 
reslice_struct = Node(Resample(voxel_size=resampled_voxel_size), 
                       name='reslice_struct')

# Registration- using FLIRT
# The BOLD image is 'in_file', the anat is 'reference', the output is 'out_file'
coreg1 = Node(FLIRT(), name='coreg1')
coreg2 = Node(FLIRT(apply_xfm=True), name = 'coreg2')

# make binary mask 
# structural is the 'in_file', output is 'binary_file'
binarize_struct = Node(Binarize(dilate=mask_dilation, min=1), 
                       name='binarize_struct')

# apply the binary mask to the functional data
# functional is 'in_file', binary mask is 'mask_file', output is 'out_file'
mask_func = Node(maths.ApplyMask(), name='mask_func')

# Spatial smoothing (FWHM 4-mm) using FSL
# Brightness threshold should be 0.75 * the contrast between the median brain intensity and the background
# inputs include in_files and mask_file
smooth = create_susan_smooth()

In [ ]:
# Function to determine brightness threshold
def brightthresh(func):
    import os
    import nibabel as nib
    from numpy import median, where
    
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    func_nifti1 = nib.load(func)
    func_data = func_nifti1.get_data()
    func_data = func_data.astype(float)
    
    brain_values = where(func_data > 0)
    median_thresh = median(brain_values)
    bright_thresh = 0.75 * median_thresh
    
    print(median_thresh)
    print(bright_thresh)
    

In [ ]:
func = 
brightthresh(func)

In [9]:
## Preprocessing Workflow

# workflowname.connect([(node1,node2,[('node1output','node2input')]),
#                    (node2,node3,[('node2output','node3input')])
#                    ])

preprocwf = Workflow(name='preprocwf')
preprocwf.connect([(infosource,selectfiles,[('subject_id','subject_id')]), 
                   (selectfiles,reorientfunc,[('func','in_file')]),
                   (reorientfunc,trimvols,[('out_file','in_file')]),
                   (trimvols,slicetime_correct,[('out_file','in_file')]),
                   (slicetime_correct,motion_correct,[('slice_time_corrected_file','in_file')]),
                   (motion_correct,coreg1,[('out_file','in_file')]),
                   (motion_correct,coreg2,[('out_file','in_file')]),
                   (selectfiles,reorientstruct,[('struct','in_file')]),
                   (reorientstruct,reslice_struct,[('out_file','in_file')]),
                   (reslice_struct,coreg1,[('resampled_file','reference')]),
                   (reslice_struct,coreg2,[('resampled_file','reference')]),
                   (coreg1, coreg2,[('out_matrix_file', 'in_matrix_file')]),
                   (reslice_struct, binarize_struct, [('resampled_file','in_file')]),
                   (binarize_struct,mask_func,[('binary_file','mask_file')]),
                   (coreg2,mask_func,[('out_file','in_file')]),
                   (mask_func,datasink,[('out_file','masked_func')]),
                   #(binarize_struct,datasink,[('binary_file','binary_struct')]),
                   #(infosource,smooth,[('fwhm','inputnode.fwhm')]),
                   #(binarize_struct,smooth,[('binary_file','inputnode.mask_file')]),
                   #(coreg,smooth,[('out_file','inputnode.in_files')]),
                   #(smooth,datasink,[('outputnode.smoothed_files','smoothed_funct')]),
                   #(trimvols,datasink,[('out_file','trimmed')]),
                   #(motion_correct,datasink,[('out_file','motion_corrected')]),
                   #(reorientstruct,datasink,[('out_file','reoriented_struct')]),
                   (reslice_struct,datasink,[('resampled_file','resliced_struct')]),
                   #(coreg,datasink,[('warped_image','registered_func')])
                   (coreg2,datasink,[('out_file','registered_func')])
                  ])
preprocwf.base_dir = output_dir
preprocwf.write_graph(graph2use='flat')
preprocwf.run('MultiProc', plugin_args={'n_procs': proc_cores})

170524-15:23:41,847 workflow INFO:
	 Creating detailed dot file: /Volumes/iang/active/BABIES/BABIES_rest/pypelineDev/processed/preproc/preprocwf/graph_detailed.dot
170524-15:23:42,288 workflow INFO:
	 Creating dot file: /Volumes/iang/active/BABIES/BABIES_rest/pypelineDev/processed/preproc/preprocwf/graph.dot
170524-15:23:42,506 workflow INFO:
	 ['check', 'execution', 'logging']
170524-15:23:42,928 workflow INFO:
	 Running in parallel.
170524-15:23:42,931 workflow INFO:
	 Executing: selectfiles.a0 ID: 0
170524-15:23:42,949 workflow INFO:
	 Executing node selectfiles.a0 in dir: /Volumes/iang/active/BABIES/BABIES_rest/pypelineDev/processed/preproc/preprocwf/_subject_id_035-BABIES-T1/selectfiles
170524-15:23:43,395 workflow INFO:
	 Runtime memory and threads stats unavailable
170524-15:23:43,437 workflow INFO:
	 [Job finished] jobname: selectfiles.a0 jobid: 0
170524-15:23:43,441 workflow INFO:
	 Executing: reorientstruct.a0 ID: 1
170524-15:23:43,455 workflow INFO:
	 [Job finished] jobname:

In [ ]:
# Resting state preprocessing

#create a binary mask

bandpass = Node(TemporalFilter(), name='bandpass')